In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

from spacy.cli.train import train
import spacy


2023-04-12 15:38:22.026943: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
DATA_DIR = "/royashcenazi/downloads/training_20180910"

#### This is a very naive start, we are training the model with the same test and validation data

In [4]:
train("/Users/royashcenazi/Studies/parsigs/research/config/config.cfg",
      output_path='first_model',
      overrides={"paths.train": "/Users/royashcenazi/downloads/train_docs.spacy", 
                 "paths.dev": "/Users/royashcenazi/downloads/test_docs.spacy",
                 "training.max_epochs": 68})

✔ Created output directory: first_model
ℹ Saving to output directory: first_model
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    934.35    0.00    0.00    0.00    0.00
  3     200        243.37  254003.48    0.00    0.00    0.00    0.00
  7     400       2718.09  44274.31    0.13   50.00    0.07    0.00
 10     600       5788.58  24754.81   12.32   26.92    7.98    0.12
 14     800       7366.95  19384.70   25.32   44.62   17.67    0.25
 17    1000      26330.60  16978.85   51.70   63.10   43.80    0.52
 21    1200      22332.91  14264.97   54.36   64.53   46.96    0.54
 25    1400       9895.50  10947.13   60.93   70.72   5

In [4]:
trained_nlp = spacy.load('./example_model/model-best')

##### first outcome! 

In [5]:
inp = 'TAKE 1 TABLET  (150 MCG TOTAL) BY MOUTH two times DAILY for one week by the way patient id is 303023 and take 2 aderol every day'.lower()
[(e, e.label_) for e in trained_nlp(inp).ents]

[(1, 'Dosage'),
 (tablet, 'Form'),
 (150 mcg, 'Strength'),
 (two times daily, 'Frequency'),
 (for one week, 'Duration'),
 (aderol, 'Drug'),
 (every day, 'Frequency')]

In [151]:
inp = 'inhale 3 puffs of albuterol 2 times a day for one week patient id is 20202'
[(e, e.label_) for e in trained_nlp(inp).ents]

[(3, 'Dosage'), (puffs, 'Form'), (albuterol, 'Drug'), (2 times a day, 'Frequency'), (for one week, 'Duration')]


 We can even parse multuple dosing instructions

In [171]:
inp = 'inhale 3 puffs of albuterol 2 times a day for one week' \
      ' then 1 puff of albuterol every day for 2 months'

print([(e, e.label_) for e in trained_nlp(inp).ents])

[(3, 'Dosage'), (puffs, 'Form'), (albuterol, 'Drug'), (2 times a day, 'Frequency'), (for one week, 'Duration'), (1, 'Dosage'), (puff, 'Form'), (albuterol, 'Drug'), (every day, 'Frequency'), (for 2 months, 'Duration')]


In [187]:
inp = 'TAKE 1 TABLET (20 MG) BY MOUTH every 2 months'.lower()
print([(e, e.label_) for e in trained_nlp(inp).ents])

{
    "DurationType": "month", 
    "DurationNum": "2",
}


[(1, 'Dosage'), (tablet, 'Form'), (20 mg, 'Strength'), (every 2 months, 'Frequency')]


{'DurationType': 'month', 'DurationNum': '2'}

 There are many things to improve, one example is the frequency that can and should be parsed to an interval type and amoubt (e.g type=day, amount=2)

Also, the model only recognizes encountered features, so inputting to the model a new sentence, will not be parsed correctly. 
When we will use the pre-trained Bert model, this issue should be resolved

In [5]:
inp = 'Take 1 tablet of ibuprofen 3 times a day'


[(e, e.label_) for e in trained_nlp(inp).ents]

NameError: name 'trained_nlp' is not defined

In [179]:
inp = ' ASPIRIN 100 mg Initiate Medication IMMEDIATE RELEASE TABLET 1.5 daily'.lower()

[(e, e.label_) for e in trained_nlp(inp).ents]

[(aspirin, 'Drug'),
 (100 mg, 'Strength'),
 (tablet, 'Form'),
 (1.5, 'Dosage'),
 (daily, 'Frequency')]